---
title: "Transformers: Attention Is All You Need"
author: "richsi"
categories: [Attention, Paper, Transformer]
date: "2024-12-30"
image: "images/thumbnail.png"
---

["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) by Ashish Vaswani et al., 2017.

## Introduction
* What is the topic of this guide, and why is it important?

The transformer is an attention-based network architecture that learns context and meaning by tracking relationships in sequential data like words in a sentence. 

* What problem does this concept or paper aim to solve? 

Sequence modeling and transduction problems such as machine translation have previously been solved by recurrent neural networks (RNNs) and long short-term memory networks (LSTMs). However, due to sequential dependency, the inherent nature of RNNs prevents its training from being parallelized. 

* What are the main contributions or breakthroughs introduced?

Transformers show significant improvements in both computational efficiency as well as model performance.

* How does this fit into the broader context of the field?

This paper laid the groundwork for state-of-the-art models such as BERT and GPT which has revolutionized the natural language processing (NLP) field. Beyong NLP, transformers have successfully adapted to other machine learning domains like computer vision and reinforcement learning. 


## Methods

### Encoder Decoder Stacks

**Encoder:** 

**Decoder:**

### Scaled Dot-Product Attention

$$
\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^{T}}{\sqrt{d_{k}}})V
$$

### Multi-Head Attention

### Positional Encoding

$$
PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}}) \\
PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})
$$

## Applications and Insights

* Where and how is this concept applied in the real world?
* What problems does it solve, and what value does it provide?
* What are some examples or use cases where this has been impactful?
* What are the broader implications or insights gained from this work?
* How does this contribute to advancements in the field or industry?

## Conclusion

* What are the key takeaways or lessons from this guide?
* Why is this concept significant in the broader context of the field?
* What questions remain unanswered or open for further exploration?
* What resources or next steps can help deepen understanding?
* How can this knowledge be applied or expanded upon in practice?